<a href="https://colab.research.google.com/github/AbhishekMajhi/DeepLearning-Using-Tensorflow/blob/master/RNN/Saimese_Sentence_Similarity_using_FastTest_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Setup in Colab

In [ ]:
!pip install kaggle

In [ ]:
import os 
os.getcwd()

'/content'

In [ ]:
!mkdir .kaggle

In [ ]:
os.listdir()

['.config', '.kaggle', 'sample_data']

In [ ]:
import json
token = {"username":"steveabhishek","key":"784b75b23b20db654f0ebdffe71d14c1"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 4.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-cp36-none-any.whl size=73265 sha256=33d15be072fd049e9ce1405802ac003cd3c1b819a6f963c9a9fef0eeab9a6985
  Stored in directory: /root/.cache/pip/wheels/68/6d/9b/7a98271454edcba3b56328cbc78c037286e787d004c8afee71
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9


In [ ]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


# Downloading FastText word to vec

In [ ]:
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m

100% 1.44G/1.44G [00:22<00:00, 92.8MB/s]
100% 1.44G/1.44G [00:22<00:00, 69.9MB/s]


In [ ]:
# Its a zip file so now to need to unzip it.
import zipfile

test_file_path = '/content/{/content}/datasets/yekenot/fasttext-crawl-300d-2m/fasttext-crawl-300d-2m.zip'


with zipfile.ZipFile(test_file_path,"r") as zip_ref:
    zip_ref.extractall("/content/sample_data/datasets")

In [ ]:
# Now lets copy this FastText model to gdrive for later use!

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# coping this file to gdrive
import shutil
file_path = r'/content/{/content}/datasets/yekenot/fasttext-crawl-300d-2m/fasttext-crawl-300d-2m.zip'
target_path = r'/content/gdrive/My Drive/Datasets/crawl-300d-2m.zip'
shutil.copyfile(file_path, target_path)

'/content/gdrive/My Drive/Datasets/crawl-300d-2m.zip'

Rather than coping this file into gdrive, we copy its bin version to gdrive because this .vec format not directly usable using Gensim. 
So in further we have done something for it. Stay tuned.

# Now we are ready to go!!

In [ ]:
# Import libraries

from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda,Dense
import keras.backend as K
from tensorflow import keras
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint


In [ ]:
# unzip test.csv.zip
import zipfile

test_file_path = '/content/gdrive/My Drive/Datasets/test.csv.zip'


with zipfile.ZipFile(test_file_path,"r") as zip_ref:
    zip_ref.extractall("/content/sample_data/datasets")

In [ ]:
# Fetch datasets
import pandas as pd
train_quora = pd.read_csv('/content/gdrive/My Drive/Datasets/train.csv')
test_quora = pd.read_csv('/content/sample_data/datasets/test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_quora.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [ ]:
test_quora.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


# Data Preprocessing

In [ ]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [ ]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding

question_col = ['question1','question2']


In [ ]:
# Loading this file is such a pain so follow this procedure
from gensim.models.keyedvectors import KeyedVectors
embedding_dict = KeyedVectors.load_word2vec_format('/content/sample_data/datasets/crawl-300d-2M.vec', binary=False) 
embedding_dict.save_word2vec_format('/content/gdrive/My Drive/Datasets/crawl-300d-2M'+".bin", binary=True) 


In [ ]:
# loading word embedding model
embedding_dict = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Datasets/crawl-300d-2M'+".bin", binary=True)

In [ ]:
# Iterate over the questions only of both training and test datasets
for dataset in [train_quora,test_quora]:
    for index,row in dataset.iterrows():
        # Iterate through the text of both question of the row
         for question in question_col:
                com = []  # for question numbers representation
                for word in text_to_word_list(row[question]):
                    # Check for unwanted words
                    
                    if word in stops and word not in embedding_dict.vocab:
                        continue
                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        com.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        com.append(vocabulary[word])
                # Replace questions as word to question as number representation
                dataset.at[index,question] = com   

In [ ]:
# Creating embedding metrix                
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

In [ ]:
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in embedding_dict.vocab:
        embeddings[index] = embedding_dict.word_vec(word)

del embedding_dict


In [ ]:
train_quora.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 11, 9, 12]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 11]",0
1,1,3,4,"[1, 2, 3, 13, 14, 15, 16, 17, 18, 17, 19, 20]","[1, 21, 22, 23, 3, 24, 25, 26, 3, 15, 16, 17, ...",0
2,2,5,6,"[28, 29, 18, 30, 3, 31, 14, 32, 33, 34, 35, 36...","[28, 29, 33, 31, 39, 40, 5, 41, 42, 43]",0
3,3,7,8,"[44, 45, 18, 46, 47, 48, 28, 29, 18, 49, 50]","[51, 3, 52, 53, 54, 55, 56, 57, 54, 2, 58, 5, ...",0
4,4,9,10,"[59, 60, 61, 9, 62, 63, 64, 65, 66, 67, 68, 69...","[59, 71, 21, 72, 9, 65, 62]",0


In [ ]:
test_quora.head()

,test_id,question1,question2
0,0,"[28, 80, 3, 1667, 1241, 5476, 749, 153, 179, 4...","[44, 334, 3867, 335, 1584, 12995, 67, 216, 158..."
1,1,"[88, 18, 405, 37, 946, 11097, 229, 837, 57, 28...","[28, 218, 538, 80, 946, 11097, 1868]"
2,2,"[1, 1224, 2, 3, 199, 254, 7, 1100, 255, 95, 82...","[1, 103, 1100, 255, 7, 821]"
3,3,"[59, 822, 216, 31960]","[1, 392, 11103]"
4,4,"[28, 86006, 1225, 1208]","[28, 294, 29, 18, 1225, 1208]"


In [ ]:
# Here we took the validation size 
validation_size = 40000
train_size = len(train_quora) - validation_size  # after removing validation data its the new size of train data.

In [ ]:
# Defining X and Y for train_test_split 
X = train_quora[question_col]   # Here it will contain questions only.
Y = train_quora['is_duplicate']  # It will contain labels only.  

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size,random_state = 101)

In [ ]:
# WE need to pass our X_train data in terms of left questions and right questions. Because we have left MALSTM and right MALSTM
X_train = {'left':X_train.question1,'right':X_train.question2}
# now for validation data
X_validation = {'left':X_validation.question1,'right':X_validation.question2}

In [ ]:
# Now we convert our train data as the input that we need, its time to convert our labels to their numpy representation.
Y_train = Y_train.values
Y_validation = Y_validation.values
# zero padding
max_seq_length = 20 # maximum length of sentence in our entire dataset
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length,dtype='int32')

In [ ]:
# Here we just make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# Model

1. Since we need to **merge** our two LSTMs output using the MaLSTM similarity function, we need to learn about keras 'merge' Layer.<br>
2. The Merge layer allows us to merge elements with some built-in methods, but also supports custom methods.
3. So we can merge our left and right LSTM together.

In [ ]:
# Now let’s define the MaLSTM similarity function.

def maLSTM_similarity_fun(left,right):
    return K.exp(-K.sum(K.abs(left-right), axis = 1, keepdims= True))

In [ ]:
# Lets initialize our parameters that we will need through-out the model.

n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
num_epochs  = 50

# Now define visible layers
left_input = Input(shape= (max_seq_length,),dtype='int32')
right_input = Input(shape = (max_seq_length,), dtype = 'int32')

# Remember we have a embedding layer in  our network architecture of MaLSTM


embedding_layer = Embedding(len(embeddings),embedding_dim, weights = [embeddings], input_length = max_seq_length,trainable = False)

# Embedding verson of input 

embedding_left = embedding_layer(left_input)
embedding_right = embedding_layer(right_input)



See keras Embedding class documentation [Embedding](https://keras.io/api/layers/core_layers/embedding/).

In [ ]:
# model 
#  Since this is a siamese network, both sides share the same LSTM.

shared_lstm = LSTM(n_hidden)

left_out = shared_lstm(embedding_left)
right_out = shared_lstm(embedding_right)

# Now let's calculate Manhattan distance as we described.
malstm_distance = Lambda(lambda x: maLSTM_similarity_fun(x[0],x[1]), output_shape = lambda x: (x[0][0],1))([left_out,right_out])
#prediction = Dense(1, activation='sigmoid', name='Similarity_layer')(malstm_distance)
# Now lets build our model

model = Model([left_input,right_input], [malstm_distance])



In [ ]:
# Optimizer

optimizer = Adadelta(learning_rate= 0.001, clipnorm=gradient_clipping_norm)

In [ ]:
model.compile(optimizer=optimizer,loss = 'mean_squared_error', metrics=['accuracy'])

In [ ]:
train_start_time = time()

model.fit([X_train['left'],X_train['right']], Y_train,batch_size= batch_size,epochs= num_epochs,
          validation_data= ([X_validation['left'],X_validation['right']],Y_validation),verbose = 2)
train_end_time = time()


Epoch 1/50
5693/5693 - 35s - loss: 0.2792 - accuracy: 0.6434 - val_loss: 0.2789 - val_accuracy: 0.6397
Epoch 2/50
5693/5693 - 35s - loss: 0.2725 - accuracy: 0.6459 - val_loss: 0.2724 - val_accuracy: 0.6432
Epoch 3/50
5693/5693 - 34s - loss: 0.2664 - accuracy: 0.6489 - val_loss: 0.2664 - val_accuracy: 0.6460
Epoch 4/50
5693/5693 - 34s - loss: 0.2607 - accuracy: 0.6517 - val_loss: 0.2609 - val_accuracy: 0.6488
Epoch 5/50
5693/5693 - 35s - loss: 0.2555 - accuracy: 0.6547 - val_loss: 0.2558 - val_accuracy: 0.6525
Epoch 6/50
5693/5693 - 34s - loss: 0.2506 - accuracy: 0.6577 - val_loss: 0.2510 - val_accuracy: 0.6555
Epoch 7/50
5693/5693 - 34s - loss: 0.2461 - accuracy: 0.6608 - val_loss: 0.2466 - val_accuracy: 0.6581
Epoch 8/50
5693/5693 - 34s - loss: 0.2420 - accuracy: 0.6638 - val_loss: 0.2426 - val_accuracy: 0.6610
Epoch 9/50
5693/5693 - 34s - loss: 0.2381 - accuracy: 0.6670 - val_loss: 0.2389 - val_accuracy: 0.6639
Epoch 10/50
5693/5693 - 35s - loss: 0.2346 - accuracy: 0.6701 - val_loss:

In [ ]:
print("Training time:", train_end_time-train_start_time)
   

Training time: 1731.7946770191193


In [ ]:
model.save('/content/gdrive/My Drive/Datasets/saimese_quora_Fasttext.h5')

In [ ]:
print("train time:", t2-t)

train time: 7.225673675537109
